In [1]:
#import keras
from IPython import get_ipython
get_ipython().magic('reset -sf') 

import pickle 
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import gc
from nltk.corpus import wordnet
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pivottablejs import pivot_ui
import random 
from nltk.probability import FreqDist
gc.collect()
    

17

In [2]:
inputFile="../data/cleaned/outputFile2-0-1000000.csv" 
df = pd.read_csv(inputFile, dtype = {"primary_price" : "string"})
df = df.drop(['id'], axis=1)
#df=df[0:10000]




In [3]:
#Cleaning the dataset


#map NLTK’s POS tags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


#Lemmatize Normalization
def normalize (tokens):
    lem = WordNetLemmatizer()
    return [lem.lemmatize(token,pos=get_wordnet_pos(token)) for token in tokens]

#clean up  tokenized  data
def standardize_tokens (tokens):
    return [token.lower() for token in tokens]

# #Few regular expressions to clean up  text data
# def standardize_text(df, text_field):
#     df[text_field] = df[text_field].str.replace(r"http\S+", "")
#     df[text_field] = df[text_field].str.replace(r".com", "")
#     df[text_field] = df[text_field].str.replace(r"http", "")
#     df[text_field] = df[text_field].str.replace(r"@\S+", "")
#    # df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
#     df[text_field] = df[text_field].str.replace(r"[(),!?@\'\`\"\_\n @]", " ")
#     df[text_field] = df[text_field].str.replace(r"nan", "")
#     df[text_field] = df[text_field].str.lower()
    
#     if(text_field != 'vendor_name'):
#         df[text_field] = df[text_field].str.replace(r"[0-9]", "")
#     #tokenize text
#     df[text_field] = df[text_field].apply(word_tokenize)
    
#     #normalize text
#     df[text_field] = df[text_field].apply(normalize)
    
#     #remove stop words
#     stop_words=set(stopwords.words("english"))
 
#     removal_tokens= stop_words.union( {',', '(', ')', "'", '"', ' ', "'s", 'nan', "''", ','})
     
#     df[text_field] = df[text_field].apply(lambda x: [item for item in x if item.strip() not in removal_tokens] )


    
import string
from string import digits 

def standardize_text_v2(text):
    tokens = word_tokenize(text)
    tokens = [w.lower() for w in tokens]
    #remove_punc = str.maketrans('', '', string.punctuation)
    #stripped = [w.translate(remove_punc) for w in tokens]
    #remove_digits = str.maketrans('', '', digits) 
    #res = [w.translate(remove_digits) for w in tokens]
    #words = [word for word in stripped if word.isalpha()]
    removals= digits+ string.punctuation
    table = str.maketrans('', '', removals) 
    words = [w.translate(table) for w in tokens]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words 

In [4]:
#df.head()

In [5]:

df['title']=df['title'].astype(str)
df['product_type']=df['product_type'].astype(str)
df['description']=df['description'].astype(str)
df['store_domain']=df['store_domain'].astype(str)
df['vendor_name']=df['vendor_name'].astype(str)
df['store_product_brand_domain']=df['store_product_brand_domain'].astype(str)
 
# df['all_text_original'] = df.values.sum(axis=1)
#df['all_text_original'] = df.apply( str.join( " " ))

df['all_text_original'] = df['product_type']+" "+df['vendor_name']+" "+df['title']+" "+df['store_domain']+" "+df['store_product_brand_domain']
df['vendor_name_original'] = df['vendor_name'].str.lower()

# standardize_text(df, 'product_type')
# standardize_text(df, 'vendor_name')
# standardize_text(df, 'title')
# standardize_text(df, 'store_domain')
# standardize_text(df, 'description')
# standardize_text(df,'store_product_brand_domain')


df['product_type']= df['product_type'].apply(standardize_text_v2)
df['vendor_name']= df['vendor_name'].apply(standardize_text_v2)
df['store_domain']= df['store_domain'].apply(standardize_text_v2)
df['description']= df['description'].apply(standardize_text_v2)
df['title']= df['title'].apply(standardize_text_v2)
df['store_product_brand_domain']= df['store_product_brand_domain'].apply(standardize_text_v2)


#combine all token in all columns into one column
#df['all_tokens'] = df.values[:,['product_type','vendor_name', 'title', 'store_domain', 'description']].sum(axis=1)
df['all_tokens'] = df.values[:,0:6].sum(axis=1)
df['all_text']= df['all_tokens'].apply(lambda x: " ".join(x))

In [6]:
# source, destination 
dbfile = open('../data/processed/processed-1milliom', 'wb')  
pickle.dump(df, dbfile)                      
dbfile.close()
df.shape

(1000000, 12)

In [7]:
df


,product_type,vendor_name,title,store_domain,store_product_brand_domain,description,primary_price,full_store_product_url,all_text_original,vendor_name_original,all_tokens,all_text
0,[cookware],[hestan],"[hestan, nanobond, stainless, qtl, covered, sa...",[atlantagrillcompanycom],[hestanculinarycom],[nan],290.0,https://atlantagrillcompany.com/products/hesta...,Cookware Hestan Hestan Nanobond Stainless 3qt/...,hestan,"[cookware, hestan, hestan, nanobond, stainless...",cookware hestan hestan nanobond stainless qtl ...
1,[cookware],[hestan],"[hestan, nanobond, titanium, stockpot, quart]",[atlantagrillcompanycom],[hestanculinarycom],[nan],450.0,https://atlantagrillcompany.com/products/hesta...,Cookware Hestan Hestan Nanobond Titanium Stock...,hestan,"[cookware, hestan, hestan, nanobond, titanium,...",cookware hestan hestan nanobond titanium stock...
2,[skillets],[hestan],"[hestan, nanobond, , , open, skillet]",[dasallascom],[hestanculinarycom],[nan],189.95,https://dasallas.com/products/hestan-nanobond-...,"Skillets Hestan Hestan Nanobond 8.5"" Open Skil...",hestan,"[skillets, hestan, hestan, nanobond, , , open,...",skillets hestan hestan nanobond open skillet...
3,"[saucepans, , sauciers]",[hestan],"[hestan, nanobond, quart, covered, essential, ...",[dasallascom],[hestanculinarycom],[nan],499.95,https://dasallas.com/products/hestan-nanobond-...,Saucepans & Sauciers Hestan Hestan Nanobond 5-...,hestan,"[saucepans, , sauciers, hestan, hestan, nanobo...",saucepans sauciers hestan hestan nanobond qua...
4,[cookware],[hestan],"[hestan, probond, forged, stainless, steel, ul...",[atlantagrillcompanycom],[hestanculinarycom],[nan],799.99,https://atlantagrillcompany.com/products/hesta...,Cookware Hestan Hestan ProBond Forged Stainles...,hestan,"[cookware, hestan, hestan, probond, forged, st...",cookware hestan hestan probond forged stainles...
...,...,...,...,...,...,...,...,...,...,...,...,...
999995,[grapes],[mbg],"[south, africa, autumn, crisp, seedless, green...",[mbgcommy],[mbgcommy],[nan],39.9,https://mbg.com.my/products/south-africa-autum...,Grapes MBG South Africa Autumn Crisp Seedless ...,mbg,"[grapes, mbg, south, africa, autumn, crisp, se...",grapes mbg south africa autumn crisp seedless ...
999996,"[stone, fruits]",[mbg],"[premium, canada, cherry, , kgwoodencrate, ]",[mbgcommy],[mbgcommy],[nan],99.9,https://mbg.com.my/products/premium-canada-che...,Stone Fruits MBG Premium Canada Cherry (1KG/Wo...,mbg,"[stone, fruits, mbg, premium, canada, cherry, ...",stone fruits mbg premium canada cherry kgwood...
999997,"[mix, , match]",[mbg],"[mbg, mix, , match, combo, , blueberry, , gree...",[mbgcommy],[mbgcommy],[nan],24.9,https://mbg.com.my/products/mbg-mix-match-comb...,Mix & Match MBG MBG Mix & Match Combo - Bluebe...,mbg,"[mix, , match, mbg, mbg, mix, , match, combo, ...",mix match mbg mbg mix match combo blueberry...
999998,"[seasonal, fruits]",[mbg],"[thailand, king, lychee, , kgpack, ]",[mbgcommy],[mbgcommy],[nan],11.9,https://mbg.com.my/products/thailand-king-lych...,Seasonal Fruits MBG Thailand King Lychee (1KG/...,mbg,"[seasonal, fruits, mbg, thailand, king, lychee...",seasonal fruits mbg thailand king lychee kgpa...
